In [8]:
#importing required libraries and modules

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import math
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D

#to get the prediction probablity in decimals and not in scientific format
np.set_printoptions(suppress=True)

In [9]:
#Setting image width and height

img_height, img_width = (224, 224)
batch_size = 32


#Setting path to training set, testing set and validation set

train_data_dir = "C:/work/train_2/train"
test_data_dir = "C:/work/train_2/test"
valid_data_dir = "C:/work/train_2/val"

In [10]:
#Setting Data Augmentation and Data generator for training and validation

train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input, 
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  validation_split=0.4)

train_generator = train_datagen.flow_from_directory(
                                    train_data_dir,
                                    target_size = (img_height, img_width),
                                    batch_size = batch_size,
                                    class_mode='categorical',
                                    subset='training')

valid_generator = train_datagen.flow_from_directory(
                                    valid_data_dir,
                                    target_size = (img_height, img_width),
                                    batch_size = batch_size,
                                    class_mode='categorical',
                                    subset='validation')

Found 304 images belonging to 10 classes.
Found 62 images belonging to 10 classes.


In [11]:
#Setting Data generator for testing 

test_generator = train_datagen.flow_from_directory(
                                    test_data_dir,
                                    target_size = (img_height, img_width),
                                    batch_size = 1,
                                    class_mode='categorical',
                                    subset='validation')

Found 67 images belonging to 10 classes.


In [12]:
x,y = test_generator.next()
x.shape

(1, 224, 224, 3)

In [13]:
#initializing and compiling Resnet50 with pretrained weights and fine tuning it by adding additional layers of Fully Connected
# Neural Network and training only those layers to optimize and adjust the entire model according to the Dog Breed Dataset

base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [14]:
#Compiling and training the Resnet50 model

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(train_generator, epochs = 10)

Epoch 1/10
10/10 [==============================] - 4s 386ms/step - loss: 1.6806 - accuracy: 0.5592
Epoch 2/10
10/10 [==============================] - 3s 277ms/step - loss: 0.2801 - accuracy: 0.9145
Epoch 3/10
10/10 [==============================] - 3s 293ms/step - loss: 0.1246 - accuracy: 0.9605
Epoch 4/10
10/10 [==============================] - 3s 295ms/step - loss: 0.0653 - accuracy: 0.9737
Epoch 5/10
10/10 [==============================] - 3s 308ms/step - loss: 0.0457 - accuracy: 0.9836
Epoch 6/10
10/10 [==============================] - 3s 292ms/step - loss: 0.0161 - accuracy: 0.9967
Epoch 7/10
10/10 [==============================] - 3s 295ms/step - loss: 0.0273 - accuracy: 0.9901
Epoch 8/10
10/10 [==============================] - 3s 308ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 3s 292ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 3s 293ms/step - loss: 0.0019 - accuracy: 1.0000

In [15]:
#testing the accuracy and loss of the newly trained Resnet50 model for dog breed classification in the test dataset created by
#splitting of the training dataset into train, test and validation dataset

test_los, test_acc = model.evaluate(test_generator, verbose = 2)

67/67 - 2s - loss: 0.1806 - accuracy: 0.9552


In [17]:
#saving the model in local storage to integrate the model into the REST API built using flask

model.save("resnet50_dog_model_4.h5")